In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime, time, sys, re

base_path = '/home/mortenhuse/EEP_projects/EEP_DataExtractor_pipeline'

if base_path not in sys.path:
    sys.path.append(base_path)
#print(sys.path)
from eep.client import GenTimeSeriesAPI, PhysicalBuildingAPI

from cognite.client import CogniteClient
from cognite.client.data_classes import AssetUpdate, LabelFilter, Relationship
from cognite.client.utils import ms_to_datetime

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
from IPython.display import display

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

In [2]:
api_key = 'M2ExY2U1ZmItNGRkNC00ZjVlLTgwMzctZDdjN2JhZDUzMjdj'
c = CogniteClient(api_key=api_key, project="energima", client_name="energima")
c.login.status()

{
    "user": "morten.huse@dev.energima.no",
    "project": "energima",
    "project_id": 7815377417292897,
    "logged_in": true,
    "api_key_id": 1338798566560387
}

In [3]:
def find_floors_from_sensordata(active_asset):
    floor_list = []
    asset_list = c.assets.list(parent_external_ids=[active_asset], limit=-1).to_pandas()
    
    if len(asset_list) > 0:
    
        for item in asset_list.externalId:
            floor = item.split('.',2)
            floor_list.append(floor[1])

        floor_list.sort()
    
    else:
        pass
    
    return floor_list

In [4]:
def find_phys_build_metadata_in_sensors(subtree_asset_extId, floor_no, sensor_filter_list = ['real_value']):
    
    list_of_assets_with_match = []
    
    active_sensor_filter_list = LabelFilter(contains_all=sensor_filter_list)
    
    asset_list_df = c.assets.list(asset_subtree_external_ids=subtree_asset_extId, labels=active_sensor_filter_list, limit=-1).to_pandas()
    
    for row in asset_list_df.itertuples():
        
        active_dict = {}
        
        active_metadata_dict = row.metadata
        
        if (active_metadata_dict.get('room_no') is not None) or (active_metadata_dict.get('room_name') is not None):

            active_dict = {
                'floor_no' : str(floor_no),
                'extId': row.externalId,
                'room_name' : active_metadata_dict.get('room_name'),
                'room_no': active_metadata_dict.get('room_no'),
                'floor': active_metadata_dict.get('floor'), #deviation from expected key name
                'object_type': active_metadata_dict.get('object_type'),
                'specie': active_metadata_dict.get('specie')   
            }
            
            list_of_assets_with_match.append(active_dict)     
        
    
    return list_of_assets_with_match, asset_list_df

In [9]:
root_asset_extId = 'AH_0459_Sagveien_17'
AAA = 'MP4'
tfm_sub_tree = '563'
active_asset = root_asset_extId + '+' + AAA + '=' + tfm_sub_tree
active_asset

'AH_0459_Sagveien_17+MP4=563'

In [10]:
#print(active_asset)

floors_list = find_floors_from_sensordata(active_asset)

sensors_in_rooms_list = []

for item in floors_list:
    
    active_asset = root_asset_extId + '+' + AAA + '=' + tfm_sub_tree + '.' + str(item)
    
    list_of_assets_with_match, asset_list_df = find_phys_build_metadata_in_sensors(subtree_asset_extId=active_asset, floor_no=item)
    
    sensors_in_rooms_list.append(list_of_assets_with_match)
    
    

In [11]:
sensors_in_rooms_list

[[],
 [{'floor_no': '001',
   'extId': 'AH_0459_Sagveien_17+MP4=563.001-RY612##CO2??B112',
   'room_name': 'Box 5',
   'room_no': 'B112',
   'floor': '1',
   'object_type': 'analog-input',
   'specie': 'co2'},
  {'floor_no': '001',
   'extId': 'AH_0459_Sagveien_17+MP4=563.001-RT609??B109',
   'room_name': 'Kopi/Miljø',
   'room_no': 'B109',
   'floor': '1',
   'object_type': 'analog-input',
   'specie': None},
  {'floor_no': '001',
   'extId': 'AH_0459_Sagveien_17+MP4=563.001-SQ409_Y??B109',
   'room_name': 'Kopi/Miljø',
   'room_no': 'B109',
   'floor': '1',
   'object_type': 'analog-input',
   'specie': None},
  {'floor_no': '001',
   'extId': 'AH_0459_Sagveien_17+MP4=563.001-RT618??B118',
   'room_name': 'Telefonrom',
   'room_no': 'B118',
   'floor': '1',
   'object_type': 'analog-input',
   'specie': None},
  {'floor_no': '001',
   'extId': 'AH_0459_Sagveien_17+MP4=563.001-SQ415-RF001??B115',
   'room_name': 'Møterom',
   'room_no': 'B115',
   'floor': '1',
   'object_type': 'anal

In [12]:
floors_rooms_tupl_list = []

for sensors_in_selected_floor in sensors_in_rooms_list:
    
    if len(sensors_in_selected_floor) > 0:
    
        rooms_tupl_list = []

        for item in sensors_in_selected_floor:

            new_unique_room_no_list = []

            #selected_tupl = (AAA, item.get('floor_no'), item.get('room_name'), item.get('room_no'))
            if len(item.get('floor')) != 3:
                selected_tupl = (AAA, item.get('floor_no'), item.get('room_name'), item.get('room_no'))
            else:
                selected_tupl = (AAA, item.get('floor'), item.get('room_name'), item.get('room_no'))
            
            print(selected_tupl)

            rooms_tupl_list.append(selected_tupl)

            new_unique_room_no_list = list(set(rooms_tupl_list))
            #new_unique_room_no_list.sort()
    
        floors_rooms_tupl_list.append(new_unique_room_no_list)

('MP4', '001', 'Box 5', 'B112')
('MP4', '001', 'Kopi/Miljø', 'B109')
('MP4', '001', 'Kopi/Miljø', 'B109')
('MP4', '001', 'Telefonrom', 'B118')
('MP4', '001', 'Møterom', 'B115')
('MP4', '001', 'Prosjektrom', 'B119')
('MP4', '001', 'Box 8', 'B117')
('MP4', '001', 'Box 3', 'B110')
('MP4', '001', 'Møterom', 'B114')
('MP4', '001', 'Box 4', 'B111')
('MP4', '001', 'Korridor', 'B105')
('MP4', '001', 'Korridor', 'B105')
('MP4', '001', 'Box 5', 'B112')
('MP4', '001', 'Kaffebar/WC', 'B101')
('MP4', '001', 'Box 4', 'B111')
('MP4', '001', 'Telefonrom', 'B118')
('MP4', '001', 'Box 4', 'B111')
('MP4', '001', 'Møterom', 'B106')
('MP4', '001', 'Møterom', 'B114')
('MP4', '001', 'Box 1', 'B107')
('MP4', '001', 'Box 4', 'B111')
('MP4', '001', 'Kaffebar/WC', 'B101')
('MP4', '001', 'Box 2', 'B108')
('MP4', '001', 'Box 6', 'B113')
('MP4', '001', 'Møterom', 'B106')
('MP4', '001', 'Prosjektrom', 'B119')
('MP4', '001', 'Kaffebar', 'B104')
('MP4', '001', 'Box 8', 'B117')
('MP4', '001', 'Møterom', 'B115')
('MP4',

In [ ]:
#floors_rooms_tupl_list[0][4] = ('YT3', 'K01', None, 'U1000')

In [13]:
for item in floors_rooms_tupl_list:
    print('')
    print('Floor: ' + str(item[0][1]))
    print('')
    room_name_list = []
    room_name_tupl_list = []
    room_name_list_duplic = []
    
    for sub in item:
        #print(sub)
        if sub[3] in room_name_list:
            #print('Duplicate: ' + str(sub))
            room_name_list_duplic.append(sub[3])
        
        active_tupl = (sub[3], sub[2])
        room_name_list.append(sub[3])
        room_name_tupl_list.append(active_tupl)
        
    print('Rooms_list: ' + str(room_name_tupl_list))
    print('Duplic_list: ' + str(room_name_list_duplic))
    
    print('===============================================')


Floor: 001

Rooms_list: [('B117', 'Box 8'), ('B119', 'Prosjektrom'), ('B104', 'Kaffebar'), ('B116', 'Box 7'), ('B106', 'Møterom'), ('B113', 'Box 6'), ('B108', 'Box 2'), ('B107', 'Box 1'), ('B111', 'Box 4'), ('B114', 'Møterom'), ('B105', 'Korridor'), ('B115', 'Møterom'), ('B101', 'Kaffebar/WC'), ('B109', 'Kopi/Miljø'), ('B110', 'Box 3'), ('B118', 'Telefonrom'), ('B112', 'Box 5')]
Duplic_list: []

Floor: 002

Rooms_list: [('221', 'WC og 211-227'), ('211', 'Klipp'), ('207A', 'Teamkontor'), ('212', 'Klipp'), ('204', 'Kontor'), ('214', 'Klipp'), ('205', 'Kontor'), ('209', 'Klipp'), ('0', None), ('201B', 'Kontor'), ('207B', 'Teamkontor'), ('203', 'Landskap'), ('202', 'Kontor'), ('216', 'Podcast'), ('201', 'Fenomen'), ('208', 'Teamrom'), ('200', 'Sosial Sone'), ('213', 'Klipp'), ('217', 'Møterom'), ('201A', 'Kontor'), ('206', 'Landskap'), ('210', 'Klipp'), ('220', 'Prosjekt/Sosial sone'), ('218', 'Møterom')]
Duplic_list: []

Floor: 003

Rooms_list: [('303', 'Tordenfilm'), ('302', 'Redbull'),

In [30]:
#manual edit
#floors_rooms_tupl_list[2][2] = ('MP4', '003', 'Maipo', '304')
#floors_rooms_tupl_list[2]
#('MP4', '002', None, '0')
#print(root_asset_extId)
#floors_rooms_tupl_list[1].remove(('MP4', '002', None, '0'))
floors_rooms_tupl_list

[[('MP4', '001', 'Box 8', 'B117'),
  ('MP4', '001', 'Prosjektrom', 'B119'),
  ('MP4', '001', 'Kaffebar', 'B104'),
  ('MP4', '001', 'Box 7', 'B116'),
  ('MP4', '001', 'Møterom', 'B106'),
  ('MP4', '001', 'Box 6', 'B113'),
  ('MP4', '001', 'Box 2', 'B108'),
  ('MP4', '001', 'Box 1', 'B107'),
  ('MP4', '001', 'Box 4', 'B111'),
  ('MP4', '001', 'Møterom', 'B114'),
  ('MP4', '001', 'Korridor', 'B105'),
  ('MP4', '001', 'Møterom', 'B115'),
  ('MP4', '001', 'Kaffebar/WC', 'B101'),
  ('MP4', '001', 'Kopi/Miljø', 'B109'),
  ('MP4', '001', 'Box 3', 'B110'),
  ('MP4', '001', 'Telefonrom', 'B118'),
  ('MP4', '001', 'Box 5', 'B112')],
 [('MP4', '002', 'WC og 211-227', '221'),
  ('MP4', '002', 'Klipp', '211'),
  ('MP4', '002', 'Teamkontor', '207A'),
  ('MP4', '002', 'Klipp', '212'),
  ('MP4', '002', 'Kontor', '204'),
  ('MP4', '002', 'Klipp', '214'),
  ('MP4', '002', 'Kontor', '205'),
  ('MP4', '002', 'Klipp', '209'),
  ('MP4', '002', 'Kontor', '201B'),
  ('MP4', '002', 'Teamkontor', '207B'),
  ('MP

In [31]:
EEP_phys_building_client = PhysicalBuildingAPI(cdf_project='energima', 
                                               root_external_id=root_asset_extId, 
                                               dryrun=False)

In [32]:
#generate rooms based on list from sensors metadata
i = 0
for floor_items in floors_rooms_tupl_list:
    for item in floor_items:
        print('---')
        print(item[0], item[1], item[2], item[3])
        
        try:
            if (len(item[2])) == 0 and (len(item[2])):
                print(item[0], item[1], item[2], item[3])
                print('Skipping too little info')
            else:
                EEP_phys_building_client.add_room(AAA=item[0], floor=str(item[1]), name=item[3],
                                                  description=item[2])

        except:
            print('error in len(), moste likely None in discr -> adding room with descr = None')
            EEP_phys_building_client.add_room(AAA=item[0], floor=str(item[1]), name=item[3], description=None)
            print(item[0], item[1], item[2], item[3])
                
            #print('---')


---
MP4 001 Box 8 B117
---
MP4 001 Prosjektrom B119
---
MP4 001 Kaffebar B104
---
MP4 001 Box 7 B116
---
MP4 001 Møterom B106
---
MP4 001 Box 6 B113
---
MP4 001 Box 2 B108
---
MP4 001 Box 1 B107
---
MP4 001 Box 4 B111
---
MP4 001 Møterom B114
---
MP4 001 Korridor B105
---
MP4 001 Møterom B115
---
MP4 001 Kaffebar/WC B101
---
MP4 001 Kopi/Miljø B109
---
MP4 001 Box 3 B110
---
MP4 001 Telefonrom B118
---
MP4 001 Box 5 B112
---
MP4 002 WC og 211-227 221
---
MP4 002 Klipp 211
---
MP4 002 Teamkontor 207A
---
MP4 002 Klipp 212
---
MP4 002 Kontor 204
---
MP4 002 Klipp 214
---
MP4 002 Kontor 205
---
MP4 002 Klipp 209
---
MP4 002 Kontor 201B
---
MP4 002 Teamkontor 207B
---
MP4 002 Landskap 203
---
MP4 002 Kontor 202
---
MP4 002 Podcast 216
---
MP4 002 Fenomen 201
---
MP4 002 Teamrom 208
---
MP4 002 Sosial Sone 200
---
MP4 002 Klipp 213
---
MP4 002 Møterom 217
---
MP4 002 Kontor 201A
---
MP4 002 Landskap 206
---
MP4 002 Klipp 210
---
MP4 002 Prosjekt/Sosial sone 220
---
MP4 002 Møterom 218
---
M

In [33]:
EEP_phys_building_client.dump_building_structure()

Building AH_0459_Sagveien_17:

#######################################################
Sub-building: MP4, Floors: 6, Metadata: None

floor: 001, rooms: 17, metadata: None

Room ID       Name            Type          Description                 Area             Metadata
----------------------------------------------------------------------------
001            B117                               Box 8                                            
002            B119                         Prosjektrom                                            
003            B104                            Kaffebar                                            
004            B116                               Box 7                                            
005            B106                             Møterom                                            
006            B113                               Box 6                                            
007            B108                               Box

In [34]:
EEP_phys_building_client.commit()

17:22:24 [Main    ] [INFO ]  Updating asset tree AH_0459_Sagveien_17
17:22:25 [AH      ] [INFO ]  Adding new asset AH_0459_Sagveien_17+MP4=2
17:22:25 [AH      ] [INFO ]  Adding new asset AH_0459_Sagveien_17+MP4=20
17:22:25 [AH      ] [INFO ]  Adding new asset AH_0459_Sagveien_17+MP4=200
17:22:25 [AH      ] [INFO ]  Adding new asset AH_0459_Sagveien_17+MP4=200.001
17:22:25 [AH      ] [INFO ]  Adding new asset AH_0459_Sagveien_17+MP4=200.001-FR001
17:22:25 [AH      ] [INFO ]  Adding new asset AH_0459_Sagveien_17+MP4=200.001-FR002
17:22:25 [AH      ] [INFO ]  Adding new asset AH_0459_Sagveien_17+MP4=200.001-FR003
17:22:25 [AH      ] [INFO ]  Adding new asset AH_0459_Sagveien_17+MP4=200.001-FR004
17:22:25 [AH      ] [INFO ]  Adding new asset AH_0459_Sagveien_17+MP4=200.001-FR005
17:22:25 [AH      ] [INFO ]  Adding new asset AH_0459_Sagveien_17+MP4=200.001-FR006
17:22:25 [AH      ] [INFO ]  Adding new asset AH_0459_Sagveien_17+MP4=200.001-FR007
17:22:25 [AH      ] [INFO ]  Adding new asset 

In [35]:
building_structure_dict = EEP_phys_building_client.list(AAA=AAA)

In [36]:
for floor_no, values in building_structure_dict[str(AAA)].items():
    
    for k,v in building_structure_dict[str(AAA)][str(floor_no)].items():
        print('k: ' + str(k))
        print('name: ' + str(v.get('name')))
        print('descr: ' + str(v.get('desc')))
        


k: 001
name: B117
descr: Box 8
k: 002
name: B119
descr: Prosjektrom
k: 003
name: B104
descr: Kaffebar
k: 004
name: B116
descr: Box 7
k: 005
name: B106
descr: Møterom
k: 006
name: B113
descr: Box 6
k: 007
name: B108
descr: Box 2
k: 008
name: B107
descr: Box 1
k: 009
name: B111
descr: Box 4
k: 010
name: B114
descr: Møterom
k: 011
name: B105
descr: Korridor
k: 012
name: B115
descr: Møterom
k: 013
name: B101
descr: Kaffebar/WC
k: 014
name: B109
descr: Kopi/Miljø
k: 015
name: B110
descr: Box 3
k: 016
name: B118
descr: Telefonrom
k: 017
name: B112
descr: Box 5
k: 001
name: 221
descr: WC og 211-227
k: 002
name: 211
descr: Klipp
k: 003
name: 207A
descr: Teamkontor
k: 004
name: 212
descr: Klipp
k: 005
name: 204
descr: Kontor
k: 006
name: 214
descr: Klipp
k: 007
name: 205
descr: Kontor
k: 008
name: 209
descr: Klipp
k: 009
name: 201B
descr: Kontor
k: 010
name: 207B
descr: Teamkontor
k: 011
name: 203
descr: Landskap
k: 012
name: 202
descr: Kontor
k: 013
name: 216
descr: Podcast
k: 014
name: 201
de

In [ ]:
#EEP_phys_building_client.cdf.assets.list()

In [37]:
floors_rooms_tupl_list

[[('MP4', '001', 'Box 8', 'B117'),
  ('MP4', '001', 'Prosjektrom', 'B119'),
  ('MP4', '001', 'Kaffebar', 'B104'),
  ('MP4', '001', 'Box 7', 'B116'),
  ('MP4', '001', 'Møterom', 'B106'),
  ('MP4', '001', 'Box 6', 'B113'),
  ('MP4', '001', 'Box 2', 'B108'),
  ('MP4', '001', 'Box 1', 'B107'),
  ('MP4', '001', 'Box 4', 'B111'),
  ('MP4', '001', 'Møterom', 'B114'),
  ('MP4', '001', 'Korridor', 'B105'),
  ('MP4', '001', 'Møterom', 'B115'),
  ('MP4', '001', 'Kaffebar/WC', 'B101'),
  ('MP4', '001', 'Kopi/Miljø', 'B109'),
  ('MP4', '001', 'Box 3', 'B110'),
  ('MP4', '001', 'Telefonrom', 'B118'),
  ('MP4', '001', 'Box 5', 'B112')],
 [('MP4', '002', 'WC og 211-227', '221'),
  ('MP4', '002', 'Klipp', '211'),
  ('MP4', '002', 'Teamkontor', '207A'),
  ('MP4', '002', 'Klipp', '212'),
  ('MP4', '002', 'Kontor', '204'),
  ('MP4', '002', 'Klipp', '214'),
  ('MP4', '002', 'Kontor', '205'),
  ('MP4', '002', 'Klipp', '209'),
  ('MP4', '002', 'Kontor', '201B'),
  ('MP4', '002', 'Teamkontor', '207B'),
  ('MP

In [38]:
#must be replaced by room externalId (from room_no) for which to use in generatation of relationships

rooms_in_floors_list = []
all_rooms_in_all_floors_list = []

for floors in floors_rooms_tupl_list:
    rooms_in_floors_list = []
    for item in floors:
        rooms_in_floors_list.append(item[3])
    
    all_rooms_in_all_floors_list.append(list(set(rooms_in_floors_list)))

In [39]:
all_rooms_in_all_floors_list

[['B107',
  'B114',
  'B105',
  'B119',
  'B110',
  'B111',
  'B101',
  'B106',
  'B104',
  'B116',
  'B113',
  'B118',
  'B109',
  'B112',
  'B117',
  'B108',
  'B115'],
 ['212',
  '214',
  '220',
  '211',
  '202',
  '221',
  '205',
  '216',
  '210',
  '217',
  '200',
  '207B',
  '213',
  '208',
  '206',
  '201B',
  '218',
  '201',
  '203',
  '201A',
  '204',
  '209',
  '207A'],
 ['304', '303', '301', '302'],
 ['401', '402'],
 ['601', '602'],
 ['702', '701']]

In [40]:
#find all floors and make list
#my_label_filter = LabelFilter(contains_all=["floor"])
my_label_filter = LabelFilter(contains_any=["floor","sub_floor"])
top_building_asset = str(root_asset_extId) + '+' + str(AAA) + '=20'
list_of_all_floors_in_building = c.assets.list(asset_subtree_external_ids=top_building_asset, labels=my_label_filter, limit=-1).to_pandas()


In [41]:
list_of_floors = list(list_of_all_floors_in_building.sort_values(by='name').name)

In [42]:
list_of_floors

['200.001', '200.002', '200.003', '200.004', '200.006', '200.007']

In [43]:
floor_nnn_list = []
for item in list_of_floors:
    floor_nnn = item.split('.')[1]
    floor_nnn_list.append(floor_nnn)

In [44]:
floor_nnn_list

['001', '002', '003', '004', '006', '007']

In [45]:
list_of_all_floors_and_rooms_in_building = []
for floor in list_of_floors:
    my_label_filter = LabelFilter(contains_all=["room"])
    top_building_asset = str(root_asset_extId) + '+' + str(AAA) + '=' + str(floor)
    list_of_all_rooms_in_building = c.assets.list(asset_subtree_external_ids=top_building_asset, labels=my_label_filter, limit=-1).to_pandas()
    list_of_all_floors_and_rooms_in_building.append(list_of_all_rooms_in_building)

In [ ]:
# active_room_no_list = list(list_of_all_floors_and_rooms_in_building[3].name)
# active_room_no_list

In [ ]:
#list_of_all_floors_and_rooms_in_building

In [46]:
dataSetId = list_of_all_floors_and_rooms_in_building[0].iloc[0].dataSetId
print(type(dataSetId))
print(dataSetId)

<class 'numpy.int64'>
3029508679944188


In [47]:
sensors_in_rooms_list

[[],
 [{'floor_no': '001',
   'extId': 'AH_0459_Sagveien_17+MP4=563.001-RY612##CO2??B112',
   'room_name': 'Box 5',
   'room_no': 'B112',
   'floor': '1',
   'object_type': 'analog-input',
   'specie': 'co2'},
  {'floor_no': '001',
   'extId': 'AH_0459_Sagveien_17+MP4=563.001-RT609??B109',
   'room_name': 'Kopi/Miljø',
   'room_no': 'B109',
   'floor': '1',
   'object_type': 'analog-input',
   'specie': None},
  {'floor_no': '001',
   'extId': 'AH_0459_Sagveien_17+MP4=563.001-SQ409_Y??B109',
   'room_name': 'Kopi/Miljø',
   'room_no': 'B109',
   'floor': '1',
   'object_type': 'analog-input',
   'specie': None},
  {'floor_no': '001',
   'extId': 'AH_0459_Sagveien_17+MP4=563.001-RT618??B118',
   'room_name': 'Telefonrom',
   'room_no': 'B118',
   'floor': '1',
   'object_type': 'analog-input',
   'specie': None},
  {'floor_no': '001',
   'extId': 'AH_0459_Sagveien_17+MP4=563.001-SQ415-RF001??B115',
   'room_name': 'Møterom',
   'room_no': 'B115',
   'floor': '1',
   'object_type': 'anal

In [52]:
BB_match_list = ['RT','RB','RY', 'RH']

In [53]:
floor_sensor_dict = {}

for floor in floor_nnn_list:
    print('')
    print(floor)
    
    list_of_sensors_in_floor = []
    for item in sensors_in_rooms_list:
        for sensor in item:
            sensor_floor = sensor.get('floor')
            if len(sensor_floor) != 3:
                sensor_floor = sensor_floor.zfill(3)
            
            print(floor)
            print(sensor_floor)
            
            if floor in sensor_floor:
                active_sensor = sensor.get('extId')
                active_sensor_room_name = sensor.get('room_no')
                active_info_tupl = (active_sensor, active_sensor_room_name)
                if any(ext in active_sensor for ext in BB_match_list):
                    print(active_info_tupl)
                    list_of_sensors_in_floor.append(active_info_tupl)

    floor_sensor_dict[str(floor)] = list_of_sensors_in_floor


001
001
001
('AH_0459_Sagveien_17+MP4=563.001-RY612##CO2??B112', 'B112')
001
001
('AH_0459_Sagveien_17+MP4=563.001-RT609??B109', 'B109')
001
001
001
001
('AH_0459_Sagveien_17+MP4=563.001-RT618??B118', 'B118')
001
001
001
001
001
001
('AH_0459_Sagveien_17+MP4=563.001-RT617??B117', 'B117')
001
001
('AH_0459_Sagveien_17+MP4=563.001-RT610??B110', 'B110')
001
001
('AH_0459_Sagveien_17+MP4=563.001-RT614??B114', 'B114')
001
001
001
001
('AH_0459_Sagveien_17+MP4=563.001-RT605??B105', 'B105')
001
001
001
001
001
001
('AH_0459_Sagveien_17+MP4=563.001-RT601??B101', 'B101')
001
001
001
001
001
001
('AH_0459_Sagveien_17+MP4=563.001-RT611??B111', 'B111')
001
001
('AH_0459_Sagveien_17+MP4=563.001-RT606??B106', 'B106')
001
001
('AH_0459_Sagveien_17+MP4=563.001-RB001??B114', 'B114')
001
001
('AH_0459_Sagveien_17+MP4=563.001-RT607??B107', 'B107')
001
001
001
001
001
001
('AH_0459_Sagveien_17+MP4=563.001-RT608??B108', 'B108')
001
001
001
001
('AH_0459_Sagveien_17+MP4=563.001-RY606##CO2??B106', 'B106')
0

001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
001
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004
002
004


In [54]:
floor_sensor_dict

{'001': [('AH_0459_Sagveien_17+MP4=563.001-RY612##CO2??B112', 'B112'),
  ('AH_0459_Sagveien_17+MP4=563.001-RT609??B109', 'B109'),
  ('AH_0459_Sagveien_17+MP4=563.001-RT618??B118', 'B118'),
  ('AH_0459_Sagveien_17+MP4=563.001-RT617??B117', 'B117'),
  ('AH_0459_Sagveien_17+MP4=563.001-RT610??B110', 'B110'),
  ('AH_0459_Sagveien_17+MP4=563.001-RT614??B114', 'B114'),
  ('AH_0459_Sagveien_17+MP4=563.001-RT605??B105', 'B105'),
  ('AH_0459_Sagveien_17+MP4=563.001-RT601??B101', 'B101'),
  ('AH_0459_Sagveien_17+MP4=563.001-RT611??B111', 'B111'),
  ('AH_0459_Sagveien_17+MP4=563.001-RT606??B106', 'B106'),
  ('AH_0459_Sagveien_17+MP4=563.001-RB001??B114', 'B114'),
  ('AH_0459_Sagveien_17+MP4=563.001-RT607??B107', 'B107'),
  ('AH_0459_Sagveien_17+MP4=563.001-RT608??B108', 'B108'),
  ('AH_0459_Sagveien_17+MP4=563.001-RY606##CO2??B106', 'B106'),
  ('AH_0459_Sagveien_17+MP4=563.001-RT619??B119', 'B119'),
  ('AH_0459_Sagveien_17+MP4=563.001-RY617##CO2??B117', 'B117'),
  ('AH_0459_Sagveien_17+MP4=563.00

In [50]:
list_of_all_floors_and_rooms_in_building[0]

,externalId,name,parentId,parentExternalId,description,dataSetId,metadata,labels,id,createdTime,lastUpdatedTime,rootId
0,AH_0459_Sagveien_17+MP4=200.001-FR012,B115,4500109252240852,AH_0459_Sagveien_17+MP4=200.001,Møterom,3029508679944188,"{'floor': '001', 'room_id': '012'}",[{'externalId': 'room'}],262709493294574,1653924145418,1653924145418,6558602457247073
1,AH_0459_Sagveien_17+MP4=200.001-FR017,B112,4500109252240852,AH_0459_Sagveien_17+MP4=200.001,Box 5,3029508679944188,"{'floor': '001', 'room_id': '017'}",[{'externalId': 'room'}],497979297127093,1653924145418,1653924145418,6558602457247073
2,AH_0459_Sagveien_17+MP4=200.001-FR001,B117,4500109252240852,AH_0459_Sagveien_17+MP4=200.001,Box 8,3029508679944188,"{'floor': '001', 'room_id': '001'}",[{'externalId': 'room'}],878342856800435,1653924145418,1653924145418,6558602457247073
3,AH_0459_Sagveien_17+MP4=200.001-FR009,B111,4500109252240852,AH_0459_Sagveien_17+MP4=200.001,Box 4,3029508679944188,"{'floor': '001', 'room_id': '009'}",[{'externalId': 'room'}],1268645742510871,1653924145418,1653924145418,6558602457247073
4,AH_0459_Sagveien_17+MP4=200.001-FR010,B114,4500109252240852,AH_0459_Sagveien_17+MP4=200.001,Møterom,3029508679944188,"{'floor': '001', 'room_id': '010'}",[{'externalId': 'room'}],3187269820875766,1653924145418,1653924145418,6558602457247073
5,AH_0459_Sagveien_17+MP4=200.001-FR014,B109,4500109252240852,AH_0459_Sagveien_17+MP4=200.001,Kopi/Miljø,3029508679944188,"{'floor': '001', 'room_id': '014'}",[{'externalId': 'room'}],3538584927764832,1653924145418,1653924145418,6558602457247073
6,AH_0459_Sagveien_17+MP4=200.001-FR003,B104,4500109252240852,AH_0459_Sagveien_17+MP4=200.001,Kaffebar,3029508679944188,"{'floor': '001', 'room_id': '003'}",[{'externalId': 'room'}],4494520363882198,1653924145418,1653924145418,6558602457247073
7,AH_0459_Sagveien_17+MP4=200.001-FR011,B105,4500109252240852,AH_0459_Sagveien_17+MP4=200.001,Korridor,3029508679944188,"{'floor': '001', 'room_id': '011'}",[{'externalId': 'room'}],4580722428510988,1653924145418,1653924145418,6558602457247073
8,AH_0459_Sagveien_17+MP4=200.001-FR008,B107,4500109252240852,AH_0459_Sagveien_17+MP4=200.001,Box 1,3029508679944188,"{'floor': '001', 'room_id': '008'}",[{'externalId': 'room'}],4746868893945796,1653924145418,1653924145418,6558602457247073
9,AH_0459_Sagveien_17+MP4=200.001-FR002,B119,4500109252240852,AH_0459_Sagveien_17+MP4=200.001,Prosjektrom,3029508679944188,"{'floor': '001', 'room_id': '002'}",[{'externalId': 'room'}],5521351677573083,1653924145418,1653924145418,6558602457247073


In [49]:

def find_room_sensor_relations(floor_sensor_dict,
                               list_of_all_floors_and_rooms_in_building_df_list,
                               BB_match_list = ['RT','RB','RY', 'RH']):
    
    relationship_dict_list = []
    relationship_cdf_list = []
    rel_extId_list = []
    floor_index = 0
    
    for rooms_in_floor_df in list_of_all_floors_and_rooms_in_building_df_list:
        
        dataSetId = rooms_in_floor_df.iloc[0].dataSetId
        
        for index, row in rooms_in_floor_df.iterrows():
            active_room_name = row['name']
            active_room_extId = row['externalId']
            active_floor_nnn = row['metadata'].get('floor')
            
            print('=======================')
            print(active_room_name, active_room_extId, active_floor_nnn)
            print('Matches:')
            print('----------------')
        
            active_floor_sensor_dict_item = floor_sensor_dict.get(active_floor_nnn)

            for sensor_extId, sensor_room_name in active_floor_sensor_dict_item:
                
                if active_room_name == sensor_room_name:
                    print('Match found!')
                    print(sensor_extId, sensor_room_name)
                    print(active_room_name, active_room_extId, active_floor_nnn)
                    
                    #creating relationships
                    
                    active_sensor_extId_ts = sensor_extId.replace('AH_','TS_')
                    
                    relationship_dict_list.append({
                                                    'floor' : str(active_floor_nnn),
                                                    'Target_name' : str(active_room_name),
                                                    'Target_extId' : str(active_room_extId),
                                                    'Source' : str(active_sensor_extId_ts),
                                                    'Source_TS' : str(active_sensor_extId_ts)
                                                  })
                    
                    rel_extId = 'rel_bt_' + str(active_room_extId) + '_<_' + str(active_sensor_extId_ts)
                    
                    #print(rel_extId)
                    
                    if rel_extId not in rel_extId_list:

                        relationship_cdf_list.append(Relationship(external_id=rel_extId,
                                         source_external_id=str(active_sensor_extId_ts),
                                         source_type="timeSeries", 
                                         target_external_id=str(active_room_extId), 
                                         target_type="asset",
                                         confidence=1,
                                         data_set_id=dataSetId)
                        )

                        rel_extId_list.append(rel_extId)
                        print('Added ' + str(rel_extId))
                    
                    else:
                        print('Rel already created: ' + str(rel_extId))
            
            
            

    
    return relationship_dict_list, relationship_cdf_list, rel_extId_list
    

In [55]:
relationship_dict_list, relationship_cdf_list, rel_extId_list = find_room_sensor_relations(floor_sensor_dict=floor_sensor_dict,
                               list_of_all_floors_and_rooms_in_building_df_list=list_of_all_floors_and_rooms_in_building,
                               BB_match_list = ['RT','RB','RY', 'RH'])

B115 AH_0459_Sagveien_17+MP4=200.001-FR012 001
Matches:
----------------
Match found!
AH_0459_Sagveien_17+MP4=563.001-RT615??B115 B115
B115 AH_0459_Sagveien_17+MP4=200.001-FR012 001
Added rel_bt_AH_0459_Sagveien_17+MP4=200.001-FR012_<_TS_0459_Sagveien_17+MP4=563.001-RT615??B115
Match found!
AH_0459_Sagveien_17+MP4=563.001-RY615##CO2??B115 B115
B115 AH_0459_Sagveien_17+MP4=200.001-FR012 001
Added rel_bt_AH_0459_Sagveien_17+MP4=200.001-FR012_<_TS_0459_Sagveien_17+MP4=563.001-RY615##CO2??B115
B112 AH_0459_Sagveien_17+MP4=200.001-FR017 001
Matches:
----------------
Match found!
AH_0459_Sagveien_17+MP4=563.001-RY612##CO2??B112 B112
B112 AH_0459_Sagveien_17+MP4=200.001-FR017 001
Added rel_bt_AH_0459_Sagveien_17+MP4=200.001-FR017_<_TS_0459_Sagveien_17+MP4=563.001-RY612##CO2??B112
Match found!
AH_0459_Sagveien_17+MP4=563.001-RT612??B112 B112
B112 AH_0459_Sagveien_17+MP4=200.001-FR017 001
Added rel_bt_AH_0459_Sagveien_17+MP4=200.001-FR017_<_TS_0459_Sagveien_17+MP4=563.001-RT612??B112
B117 AH_04

AH_0459_Sagveien_17+MP4=563.007-RY002??702 702
702 AH_0459_Sagveien_17+MP4=200.007-FR001 007
Added rel_bt_AH_0459_Sagveien_17+MP4=200.007-FR001_<_TS_0459_Sagveien_17+MP4=563.007-RY002??702
Match found!
AH_0459_Sagveien_17+MP4=563.007-RB002??702 702
702 AH_0459_Sagveien_17+MP4=200.007-FR001 007
Added rel_bt_AH_0459_Sagveien_17+MP4=200.007-FR001_<_TS_0459_Sagveien_17+MP4=563.007-RB002??702


In [56]:
len(relationship_dict_list)

109

In [57]:
len(relationship_cdf_list)

109

In [58]:
relationship_cdf_list

In [59]:
res = c.relationships.create(relationship_cdf_list)

In [60]:
res

,external_id,source_external_id,source_type,target_external_id,target_type,confidence,data_set_id,created_time,last_updated_time,labels
0,rel_bt_AH_0459_Sagveien_17+MP4=200.001-FR012_<_TS_0459_Sagveien_17+MP4=563.001-RT615??B115,TS_0459_Sagveien_17+MP4=563.001-RT615??B115,timeSeries,AH_0459_Sagveien_17+MP4=200.001-FR012,asset,1.0,3029508679944188,1653924576009,1653924576009,[]
1,rel_bt_AH_0459_Sagveien_17+MP4=200.001-FR012_<_TS_0459_Sagveien_17+MP4=563.001-RY615##CO2??B115,TS_0459_Sagveien_17+MP4=563.001-RY615##CO2??B115,timeSeries,AH_0459_Sagveien_17+MP4=200.001-FR012,asset,1.0,3029508679944188,1653924576009,1653924576009,[]
2,rel_bt_AH_0459_Sagveien_17+MP4=200.001-FR017_<_TS_0459_Sagveien_17+MP4=563.001-RY612##CO2??B112,TS_0459_Sagveien_17+MP4=563.001-RY612##CO2??B112,timeSeries,AH_0459_Sagveien_17+MP4=200.001-FR017,asset,1.0,3029508679944188,1653924576009,1653924576009,[]
3,rel_bt_AH_0459_Sagveien_17+MP4=200.001-FR017_<_TS_0459_Sagveien_17+MP4=563.001-RT612??B112,TS_0459_Sagveien_17+MP4=563.001-RT612??B112,timeSeries,AH_0459_Sagveien_17+MP4=200.001-FR017,asset,1.0,3029508679944188,1653924576009,1653924576009,[]
4,rel_bt_AH_0459_Sagveien_17+MP4=200.001-FR001_<_TS_0459_Sagveien_17+MP4=563.001-RT617??B117,TS_0459_Sagveien_17+MP4=563.001-RT617??B117,timeSeries,AH_0459_Sagveien_17+MP4=200.001-FR001,asset,1.0,3029508679944188,1653924576009,1653924576009,[]
5,rel_bt_AH_0459_Sagveien_17+MP4=200.001-FR001_<_TS_0459_Sagveien_17+MP4=563.001-RY617##CO2??B117,TS_0459_Sagveien_17+MP4=563.001-RY617##CO2??B117,timeSeries,AH_0459_Sagveien_17+MP4=200.001-FR001,asset,1.0,3029508679944188,1653924576009,1653924576009,[]
6,rel_bt_AH_0459_Sagveien_17+MP4=200.001-FR009_<_TS_0459_Sagveien_17+MP4=563.001-RT611??B111,TS_0459_Sagveien_17+MP4=563.001-RT611??B111,timeSeries,AH_0459_Sagveien_17+MP4=200.001-FR009,asset,1.0,3029508679944188,1653924576009,1653924576009,[]
7,rel_bt_AH_0459_Sagveien_17+MP4=200.001-FR009_<_TS_0459_Sagveien_17+MP4=563.001-RY611##CO2??B111,TS_0459_Sagveien_17+MP4=563.001-RY611##CO2??B111,timeSeries,AH_0459_Sagveien_17+MP4=200.001-FR009,asset,1.0,3029508679944188,1653924576009,1653924576009,[]
8,rel_bt_AH_0459_Sagveien_17+MP4=200.001-FR010_<_TS_0459_Sagveien_17+MP4=563.001-RT614??B114,TS_0459_Sagveien_17+MP4=563.001-RT614??B114,timeSeries,AH_0459_Sagveien_17+MP4=200.001-FR010,asset,1.0,3029508679944188,1653924576009,1653924576009,[]
9,rel_bt_AH_0459_Sagveien_17+MP4=200.001-FR010_<_TS_0459_Sagveien_17+MP4=563.001-RB001??B114,TS_0459_Sagveien_17+MP4=563.001-RB001??B114,timeSeries,AH_0459_Sagveien_17+MP4=200.001-FR010,asset,1.0,3029508679944188,1653924576009,1653924576009,[]


In [61]:
rel_df = c.relationships.list(data_set_ids=[3029508679944188], limit=-1).to_pandas()
rel_df

,externalId,sourceExternalId,sourceType,targetExternalId,targetType,confidence,dataSetId,createdTime,lastUpdatedTime,labels
0,rel_bt_AH_0459_Sagveien_17+MP4=200.001-FR012_<_TS_0459_Sagveien_17+MP4=563.001-RT615??B115,TS_0459_Sagveien_17+MP4=563.001-RT615??B115,timeSeries,AH_0459_Sagveien_17+MP4=200.001-FR012,asset,1.0,3029508679944188,1653924576009,1653924576009,[]
1,rel_bt_AH_0459_Sagveien_17+MP4=200.001-FR012_<_TS_0459_Sagveien_17+MP4=563.001-RY615##CO2??B115,TS_0459_Sagveien_17+MP4=563.001-RY615##CO2??B115,timeSeries,AH_0459_Sagveien_17+MP4=200.001-FR012,asset,1.0,3029508679944188,1653924576009,1653924576009,[]
2,rel_bt_AH_0459_Sagveien_17+MP4=200.001-FR017_<_TS_0459_Sagveien_17+MP4=563.001-RY612##CO2??B112,TS_0459_Sagveien_17+MP4=563.001-RY612##CO2??B112,timeSeries,AH_0459_Sagveien_17+MP4=200.001-FR017,asset,1.0,3029508679944188,1653924576009,1653924576009,[]
3,rel_bt_AH_0459_Sagveien_17+MP4=200.001-FR017_<_TS_0459_Sagveien_17+MP4=563.001-RT612??B112,TS_0459_Sagveien_17+MP4=563.001-RT612??B112,timeSeries,AH_0459_Sagveien_17+MP4=200.001-FR017,asset,1.0,3029508679944188,1653924576009,1653924576009,[]
4,rel_bt_AH_0459_Sagveien_17+MP4=200.001-FR001_<_TS_0459_Sagveien_17+MP4=563.001-RT617??B117,TS_0459_Sagveien_17+MP4=563.001-RT617??B117,timeSeries,AH_0459_Sagveien_17+MP4=200.001-FR001,asset,1.0,3029508679944188,1653924576009,1653924576009,[]
5,rel_bt_AH_0459_Sagveien_17+MP4=200.001-FR001_<_TS_0459_Sagveien_17+MP4=563.001-RY617##CO2??B117,TS_0459_Sagveien_17+MP4=563.001-RY617##CO2??B117,timeSeries,AH_0459_Sagveien_17+MP4=200.001-FR001,asset,1.0,3029508679944188,1653924576009,1653924576009,[]
6,rel_bt_AH_0459_Sagveien_17+MP4=200.001-FR009_<_TS_0459_Sagveien_17+MP4=563.001-RT611??B111,TS_0459_Sagveien_17+MP4=563.001-RT611??B111,timeSeries,AH_0459_Sagveien_17+MP4=200.001-FR009,asset,1.0,3029508679944188,1653924576009,1653924576009,[]
7,rel_bt_AH_0459_Sagveien_17+MP4=200.001-FR009_<_TS_0459_Sagveien_17+MP4=563.001-RY611##CO2??B111,TS_0459_Sagveien_17+MP4=563.001-RY611##CO2??B111,timeSeries,AH_0459_Sagveien_17+MP4=200.001-FR009,asset,1.0,3029508679944188,1653924576009,1653924576009,[]
8,rel_bt_AH_0459_Sagveien_17+MP4=200.001-FR010_<_TS_0459_Sagveien_17+MP4=563.001-RT614??B114,TS_0459_Sagveien_17+MP4=563.001-RT614??B114,timeSeries,AH_0459_Sagveien_17+MP4=200.001-FR010,asset,1.0,3029508679944188,1653924576009,1653924576009,[]
9,rel_bt_AH_0459_Sagveien_17+MP4=200.001-FR010_<_TS_0459_Sagveien_17+MP4=563.001-RB001??B114,TS_0459_Sagveien_17+MP4=563.001-RB001??B114,timeSeries,AH_0459_Sagveien_17+MP4=200.001-FR010,asset,1.0,3029508679944188,1653924576009,1653924576009,[]


In [ ]:
test_list = rel_df[rel_df['sourceExternalId'].str.contains('-RH')]
test_list_sub = test_list['targetExternalId'].to_list()
test_list_unique = list(set(test_list_sub))
print(len(test_list))
print(len(test_list_unique))
test_list_unique

In [ ]:
count = 0
count_exeption = 0
rel_room_sensor_del_list = []
rel_other_del_list = []

for index, row in rel_df.iterrows():
    extId = row['externalId']
    date_create = ms_to_datetime(row['createdTime'])
    if date_create < (datetime.datetime.utcnow() - datetime.timedelta(hours=2)):
        #print(extId, date_create)
        
        if (extId.find('-FR') != -1):
            rel_room_sensor_del_list.append(extId)
            count +=1
        else:
            #print('NO match!')
            #print(extId, date_create)
            rel_other_del_list.append(extId)
            count_exeption +=1

In [ ]:
len(rel_room_sensor_del_list)

In [ ]:
len(rel_other_del_list)

In [ ]:
c.relationships.delete(external_id=rel_other_del_list)

In [ ]:
c.relationships.retrieve(external_id='rel_sp_rv_heating_system_TS_0181_Youngstorget_3+YT3=320.002-RT926_Xs_0002')

In [ ]:
c.time_series.retrieve(external_id='TS_0181_Youngstorget_3+YT3=320.002-RT926_Xs')

In [ ]:
c.time_series.retrieve(external_id='TS_0181_Youngstorget_3+YT3=320.002-RT502')

In [ ]:
#             for room_name in active_room_no_list:

#                 if room_name == active_sensor_room_name:
#                     index_match = active_room_no_list.index(room_name)
#                     room_extId = str(active_room_extId_list[index_match])
#                     active_sensor_extId_ts = active_sensor_extId.replace('AH_','TS_')

#                     relationship_dict_list.append({
#                                                     'floor' : str(sensor.get('floor_no')),
#                                                     'Target_name' : str(room_name),
#                                                     'Target_extId' : str(room_extId),
#                                                     'Source' : str(sensor.get('extId')),
#                                                     'Source_TS' : active_sensor_extId_ts
#                                                   })



#                     rel_extId = 'rel_bt_' + str(room_extId) + '_<_' + str(active_sensor_extId_ts)
#                     print(rel_extId)

#                     if rel_extId not in rel_extId_list:

#                         relationship_cdf_list.append(Relationship(external_id=rel_extId,
#                                          source_external_id=str(active_sensor_extId_ts),
#                                          source_type="timeSeries", 
#                                          target_external_id=str(room_extId), 
#                                          target_type="asset",
#                                          confidence=1,
#                                          data_set_id=dataSetId)
#                         )

#                         rel_extId_list.append(rel_extId)

#                     else:

#                         print('Rel already created: ' + rel_extId)
                      


# ==================================================================================
    
    
#     for sensors_in_selected_floor in sensors_in_rooms_list:
        
#         print('Floor: ' + str(floor_index))
        
#         active_room_no_list = list(list_of_all_rooms_in_building_df[floor_index].name)
#         print(active_room_no_list)
#         active_room_extId_list = list(list_of_all_rooms_in_building_df[floor_index].externalId)
        
#         for sensor in sensors_in_selected_floor:
            
#             active_sensor_room_name = sensor.get('room_no')
#             active_sensor_extId = sensor.get('extId')
            
#             print(active_sensor_room_name)
#             print(active_sensor_extId)
            
            
# #             if active_sensor_extId in BB_match_list:
# #                 print('Sensor match BB: ' + active_sensor_extId)
            
#             if active_sensor_room_name is not None:

#                 for room_name in active_room_no_list:

#                     if room_name == active_sensor_room_name:
#                         index_match = active_room_no_list.index(room_name)
#                         room_extId = str(active_room_extId_list[index_match])
#                         active_sensor_extId_ts = active_sensor_extId.replace('AH_','TS_')

#                         relationship_dict_list.append({
#                                                         'floor' : str(sensor.get('floor_no')),
#                                                         'Target_name' : str(room_name),
#                                                         'Target_extId' : str(room_extId),
#                                                         'Source' : str(sensor.get('extId')),
#                                                         'Source_TS' : active_sensor_extId_ts
#                                                       })



#                         rel_extId = 'rel_bt_' + str(room_extId) + '_<_' + str(active_sensor_extId_ts)
#                         print(rel_extId)

#                         if rel_extId not in rel_extId_list:

#                             relationship_cdf_list.append(Relationship(external_id=rel_extId,
#                                              source_external_id=str(active_sensor_extId_ts),
#                                              source_type="timeSeries", 
#                                              target_external_id=str(room_extId), 
#                                              target_type="asset",
#                                              confidence=1,
#                                              data_set_id=dataSetId)
#                             )

#                             rel_extId_list.append(rel_extId)

#                         else:

#                             print('Rel already created: ' + rel_extId)


# #             else:
# #                 print('Skipping sensor due to non-matching BB: ' + active_sensor_extId)

        
#         floor_index +=1

In [3]:
test_client = PhysicalBuildingAPI(cdf_project='energima',root_external_id='AH_0277_Drammensveien_133')

In [ ]:
test_client.add_room()